In [1]:
import pandas as pd
from pathlib import Path
from semif_utils import get_images, save_metadata, read_metadata
import dataclasses
from tqdm import tqdm
from datasets import BatchMetadata, ImageData, EnvironmentalMetadata, BatchImageList

In [2]:
from chinmay_dummy_data.bbox.connectors import SfMComponents, BBoxComponents
from chinmay_dummy_data.bbox.bbox_transformations import BBoxMapper, BBoxFilter
from chinmay_dummy_data.bbox.io_utils import ParseXML
import os

In [3]:
image_dir = "data/semif-trial"
batch_metadata = Path(image_dir, "batch_metadata.yaml")

In [4]:
bmeta = read_batch_metadata(batch_metadata)
env_meta = EnvironmentalMetadata(site_id=bmeta["location"])
batch_meta = BatchMetadata(site_id=bmeta["location"],upload_dir=image_dir, upload_datetime=bmeta["date"], env_metadata=env_meta)
images = BatchImageList( batch_metadata=batch_meta)

In [5]:
base_path = "chinmay_dummy_data/exports/semif-trial/reference"
connector = SfMComponents(base_path)
gcp_reference = connector.gcp_reference
camera_reference = connector.camera_reference
image_path = "chinmay_dummy_data/exports/semif-trial/semif-trial/"
orthomosaic_path = "chinmay_dummy_data/exports/semif-trial/images/orthomosaic.tif"
annotation_file_path = "chinmay_dummy_data/exports/semif-trial/annotations"
metadata_save_path = "chinmay_dummy_data/exports/semif-trial/metadata"
if not os.path.exists(metadata_save_path):
    os.makedirs(metadata_save_path)

In [6]:
# Initialize the reader which will read the annotation files and convert the bounding
# boxes to the desired format
reader = ParseXML(image_path=image_path, label_path=annotation_file_path)

In [7]:
# Initialize the connector and get a list of all the images
box_connector = BBoxComponents(camera_reference, reader, image_path, annotation_file_path)
boxes = box_connector.bboxes


In [8]:
images = get_images(image_dir, sort=True)
image_ids = [image.stem for image in images]
image_list = [{
            "id": image_id,
            "path": str(path)
        } for image_id, path in zip(image_ids, images)]
imgs = []
for img in image_list:
    id = img["id"]
    path = img["path"]
    bboxes = boxes[id]
    img = ImageData(image_id=id, image_path=path, upload_id=batch_meta.upload_id, bboxes=bboxes)
    imgs.append(img)

    

In [11]:
ex = dataclasses.asdict(imgs[20])
ex

# save_metadata(ex, "test.json")

{'image_id': 'row4_3',
 'image_path': 'data/semif-trial/row4_3.jpg',
 'batch_metadata': {'site_id': 'tx',
  'upload_datetime': datetime.date(2022, 5, 6),
  'upload_dir': 'data/semif-trial',
  'env_metadata': {'site_id': 'tx',
   'hardiness_zone': '8b',
   'humidity': None,
   'precipitation': None},
  'upload_id': 'ab694b63-9748-4c6e-bfb4-dabcef787a67',
  'schema_version': 'v1'},
 'bboxes': [{'id': 'row4_3_0',
   'image_id': 'row4_3',
   'cls': 'target 3',
   'local_coordinates': {'top_left': array([344.,   1.]),
    'top_right': array([1.694e+03, 1.000e+00]),
    'bottom_left': array([344., 867.]),
    'bottom_right': array([1694.,  867.])},
   'global_coordinates': {'top_left': array([], dtype=float64),
    'top_right': array([], dtype=float64),
    'bottom_left': array([], dtype=float64),
    'bottom_right': array([], dtype=float64)},
   'local_centroid': array([1019.,  434.]),
   'global_centroid': array([], dtype=float64),
   'is_primary': False},
  {'id': 'row4_3_1',
   'image_id

In [31]:
import inspect
print(type(boxes))
# inspect.getmembers(box_connector, lambda a:not(inspect.isroutine(a)))

<class 'dict'>
